**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
from tensorflow import config

config.list_physical_devices('GPU')

2025-08-04 12:13:40.166474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754309620.311449      32 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754309620.353222      32 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754309620.685215      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309620.685296      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309620.685299      32 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 본다.

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend, layers
from tensorflow import keras
import tensorflow as tf
from pathlib import Path

mnist_ = mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist_
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
backend.clear_session()
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.export(model_path)

I0000 00:00:1754309624.853464      32 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5557 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1754309626.224297     101 service.cc:152] XLA service 0x7778ac004e70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754309626.224333     101 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-08-04 12:13:46.259546: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754309626.325360     101 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-04 12:13:47.023906: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36_0', 92 bytes spill stores, 96 bytes spill loads

2025-08-04 12:13:47.157050: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36

  79/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1827 - loss: 2.2922

I0000 00:00:1754309629.381301     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6963 - loss: 1.1423

2025-08-04 12:13:57.484920: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-04 12:13:57.521570: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 296 bytes spill stores, 296 bytes spill loads

2025-08-04 12:13:57.599558: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 792 bytes spill stores, 844 bytes spill loads

2025-08-04 12:13:57.694321: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-04 12:13:57.870421: I external/local_xla

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.6963 - loss: 1.1420 - val_accuracy: 0.9026 - val_loss: 0.3724
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8995 - loss: 0.3653 - val_accuracy: 0.9206 - val_loss: 0.3000
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9133 - loss: 0.3096 - val_accuracy: 0.9282 - val_loss: 0.2663
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9219 - loss: 0.2783 - val_accuracy: 0.9342 - val_loss: 0.2409
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9290 - loss: 0.2527 - val_accuracy: 0.9396 - val_loss: 0.2242
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9353 - loss: 0.2314 - val_accuracy: 0.9442 - val_loss: 0.2086
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9404 - loss: 0.2163 - val_accuracy: 0.9472 - val_loss: 0.1972
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9423 - loss: 0.2057 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  131364139823952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364139825104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364139822416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364139825680: TensorSpec(shape=(), dtype=tf.resource, name=None)


파일 트리를 살핀다:

In [3]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [4]:
!saved_model_cli show --dir '{model_path}'

2025-08-04 12:14:32.800366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754309672.813313     914 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754309672.818577     914 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754309672.833539     914 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309672.833574     914 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309672.833580     914 computation_placer.cc:177] computation placer alr

In [5]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2025-08-04 12:14:35.677952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754309675.692005     930 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754309675.696145     930 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754309675.707853     930 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309675.707889     930 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309675.707895     930 computation_placer.cc:177] computation placer alr

In [6]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2025-08-04 12:14:38.127144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754309678.140156     946 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754309678.144184     946 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754309678.155970     946 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309678.156005     946 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754309678.156012     946 computation_placer.cc:177] computation placer alr

더 자세한 내용을 보려면 다음 명령을 실행한다:
```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다.
```bash
docker pull tensorflow/serving:latest-gpu

docker run -e MODEL_NAME=my_mnist_model --gpus all -it -p 8500:8500 -p 8501:8501 -v path\to\my_mnist_model:/models/my_mnist_model tensorflow/serving:latest-gpu
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송한다:

In [7]:
import json

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정한다.
request_json = json.dumps({'signature_name': 'serving_default', 'instances': X_new.tolist()})

In [8]:
request_json[:100] + '...' + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측한다:

In [9]:
import requests

server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [10]:
import numpy as np

y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [11]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = 'keras_tensor'
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [12]:
request

model_spec {
  name: "my_mnist_model"
  signature_name: "serving_default"
}
inputs {
  key: "keras_tensor"
  value {
    dtype: DT_UINT8
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
    }
    tensor_content: "\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\00

In [13]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('172.17.0.3:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환한다:

In [14]:
output_name = 'output_0'
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [15]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

### 새 모델 버전 배포

In [16]:
# 새로운 MNIST 모델 버전 빌드 및 훈련
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10


2025-08-04 12:14:43.205076: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-04 12:14:43.205142: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 488 bytes spill stores, 488 bytes spill loads

2025-08-04 12:14:43.508522: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 940 bytes spill stores, 940 bytes spill loads

2025-08-04 12:14:43.511192: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-08-04 12:14:43.642365: I external/l

1710/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6187 - loss: 1.2799

2025-08-04 12:14:49.229960: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-04 12:14:49.609145: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 988 bytes spill stores, 988 bytes spill loads

2025-08-04 12:14:49.721077: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 956 bytes spill stores, 956 bytes spill loads

2025-08-04 12:14:49.826971: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 484 bytes spill stores, 484 bytes spill loads

2025-08-04 12:14:50.153736: I external/l

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6195 - loss: 1.2773

2025-08-04 12:14:53.065450: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-04 12:14:53.101673: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 348 bytes spill stores, 348 bytes spill loads

2025-08-04 12:14:53.229334: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-04 12:14:53.310106: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 816 bytes spill stores, 816 bytes spill loads



1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.6196 - loss: 1.2770 - val_accuracy: 0.9000 - val_loss: 0.3551
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8966 - loss: 0.3567 - val_accuracy: 0.9262 - val_loss: 0.2742
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9188 - loss: 0.2851 - val_accuracy: 0.9332 - val_loss: 0.2394
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9289 - loss: 0.2434 - val_accuracy: 0.9404 - val_loss: 0.2119
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9379 - loss: 0.2164 - val_accuracy: 0.9456 - val_loss: 0.1958
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9423 - loss: 0.2018 - val_accuracy: 0.9476 - val_loss: 0.1786
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9484 - loss: 0.1840 - val_accuracy: 0.9522 - val_loss: 0.1680
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9505 - loss: 0.1707 - val_accura

In [17]:
model_version = '0002'
model_path = Path(model_name) / model_version
model.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Saved artifact at 'my_mnist_model/0002'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  131364097278032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097279952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364139816272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097283984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097283600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097283408: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [18]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있다.

In [19]:
server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()
response = response.json()

In [20]:
response.keys()

dict_keys(['predictions'])

In [21]:
y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

# 모바일 또는 임베디드 디바이스에 모델 배포하기

In [22]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open('my_converted_savedmodel.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1754309733.849280      32 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1754309733.849327      32 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-04 12:15:33.852709: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: my_mnist_model/0002
2025-08-04 12:15:33.855694: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-04 12:15:33.855704: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: my_mnist_model/0002
I0000 00:00:1754309733.859936      32 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-08-04 12:15:33.860535: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-04 12:15:33.883713: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: my_mnist_model/0002
2025-08-04 12:15:33.888835: I tensorflow/cc/saved_model/loader.cc:471] Sa

In [23]:
# 케라스 모델을 변환한다.
converter = lite.TFLiteConverter.from_keras_model(model)

In [24]:
converter.optimizations = [lite.Optimize.DEFAULT]

In [25]:
tflite_model = converter.convert()
with open('my_converted_keras_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpmfs9z154/assets


INFO:tensorflow:Assets written to: /tmp/tmpmfs9z154/assets


Saved artifact at '/tmp/tmpmfs9z154'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  131364097278032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097279952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364139816272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097283984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097283600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131364097283408: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1754309734.153280      32 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1754309734.153317      32 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-04 12:15:34.153444: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpmfs9z154
2025-08-04 12:15:34.153785: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-04 12:15:34.153792: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpmfs9z154
2025-08-04 12:15:34.156361: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-04 12:15:34.171053: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpmfs9z154
2025-08-04 12:15:34.175481: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 22039 microseconds.


# GPU를 사용하여 계산 속도 향상하기

텐서플로우가 GPU를 볼 수 있는지 확인한다:

In [27]:
physical_gpus = config.list_physical_devices('GPU')
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

텐서플로 스크립트에서 GPU \#0과 \#1(PCI 순서 기준)만 사용하려면 스크립트를 시작하기 전에 환경 변수 `CUDA_DEVICE_ORDER=PCI_BUS_ID`와 `CUDA_VISIBLE_DEVICES=0,1`을 설정한다. 또는 스크립트 자체에서 텐서플로를 사용하기 전에 지정한다.

## GPU RAM 관리

RAM 용량을 GPU당 2GB로 제한하려면:

In [42]:
# for gpu in physical_gpus:
#     config.set_logical_device_configuration(gpu, [config.LogicalDeviceConfiguration(2048)])

점진적으로 텐서플로가 메모리를 점유하도록 하려면(프로세스가 종료될 때만 메모리를 해제한다):

In [44]:
# from tensorflow.config import experimental

# for gpu in physical_gpus:
#     experimental.set_memory_growth(gpu, True)